In [1]:
import pandas as pd
from simpletransformers.classification import ClassificationModel
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
import os
import torch

In [2]:
kfold = 10
train_folds_df = pd.read_csv("deneme_son.csv", sep="|", encoding="utf-8")
train_folds_df.head()

,tweet,target
0,yahu internetim de sorun var turkcell tedas ye...,olumsuz
1,lifebox da eski mailim kayıtlı oldugu icin gir...,olumsuz
2,turkcell tv hizmetinizi kontrol edebilmemiz ic...,nötr
3,sevgili turkcell twitterı ucretsiz kullanıyors...,olumsuz
4,turkcell beni magdur ediyor kaan bey kaanterzi...,olumsuz


In [3]:
if 'kfold' not in train_folds_df.columns:
    kf = KFold(n_splits=kfold, shuffle=True, random_state=42)
    train_folds_df['kfold'] = -1
    for fold, (train_index, val_index) in enumerate(kf.split(train_folds_df)):
        train_folds_df.loc[val_index, 'kfold'] = fold

In [4]:
train_folds_df.head()

,tweet,target,kfold
0,yahu internetim de sorun var turkcell tedas ye...,olumsuz,2
1,lifebox da eski mailim kayıtlı oldugu icin gir...,olumsuz,7
2,turkcell tv hizmetinizi kontrol edebilmemiz ic...,nötr,5
3,sevgili turkcell twitterı ucretsiz kullanıyors...,olumsuz,1
4,turkcell beni magdur ediyor kaan bey kaanterzi...,olumsuz,7


In [5]:
train_folds_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22343 entries, 0 to 22342
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   22342 non-null  object
 1   target  22343 non-null  object
 2   kfold   22343 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 523.8+ KB


In [6]:
le = LabelEncoder()
le.fit(train_folds_df['target'])
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
le_name_mapping = {k: int(v) for k, v in le_name_mapping.items()}
print(le_name_mapping)
train_folds_df['labels'] = train_folds_df['target'].apply(lambda x: le_name_mapping[x])

{'nötr': 0, 'olumlu': 1, 'olumsuz': 2}


In [7]:
train_folds_df.head(10)

,tweet,target,kfold,labels
0,yahu internetim de sorun var turkcell tedas ye...,olumsuz,2,2
1,lifebox da eski mailim kayıtlı oldugu icin gir...,olumsuz,7,2
2,turkcell tv hizmetinizi kontrol edebilmemiz ic...,nötr,5,0
3,sevgili turkcell twitterı ucretsiz kullanıyors...,olumsuz,1,2
4,turkcell beni magdur ediyor kaan bey kaanterzi...,olumsuz,7,2
5,turkcell reklamı gibisin oyle bos oyle anlamsı...,olumsuz,3,2
6,turkcell star fizy istemedigim halde uye yapıl...,olumsuz,2,2
7,turkcell kullanma yın hayatımın en buyuk hatas...,olumsuz,5,2
8,turkcell gne kuredecem senin ulan bir gunde gb...,olumsuz,3,2
9,turkcell boyut atladı artık hic cekmiyorintern...,olumsuz,9,2


In [8]:
for fold in range(kfold):
    # Split data into train and validation sets
    train_df = train_folds_df[train_folds_df['kfold'] != fold]
    validation_df = train_folds_df[train_folds_df['kfold'] == fold]

    # Prepare the data for simpletransformers
    train_df = pd.DataFrame({
        'text': train_df['tweet'],  # Replace 'your_text_column' with the actual column name for text
        'labels': train_df['labels']
    })
    validation_df = pd.DataFrame({
        'text': validation_df['tweet'],  # Replace 'your_text_column' with the actual column name for text
        'labels': validation_df['labels']
    })

    # Initialize the ClassificationModel
    model = ClassificationModel('bert',
                                'dbmdz/bert-base-turkish-cased',
                                num_labels=len(le_name_mapping.keys()),
                                use_cuda=True,  # Use CUDA
                                args={'reprocess_input_data': True,
                                      'overwrite_output_dir': True,
                                      'num_train_epochs': 12,
                                      'train_batch_size': 64,
                                      'fp16': False,
                                      'save_model_every_epoch': True,
                                      'save_eval_checkpoints': False,
                                      'output_dir': f'bert_model_fold_merge_{fold}',
                                      'save_steps': 0 })

/home/ensar/anaconda3/lib/python3.9/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at dbmdz/bert-base-turkish-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- 

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at dbmdz/bert-base-turkish-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining

In [9]:
model.train_model(train_df)

  0%|          | 0/20109 [00:00<?, ?it/s]

Epoch:   0%|          | 0/12 [00:00<?, ?it/s]

Running Epoch 0 of 12:   0%|          | 0/315 [00:00<?, ?it/s]

Running Epoch 1 of 12:   0%|          | 0/315 [00:00<?, ?it/s]

Running Epoch 2 of 12:   0%|          | 0/315 [00:00<?, ?it/s]

...

Running Epoch 8 of 12:   0%|          | 0/315 [00:00<?, ?it/s]

Running Epoch 9 of 12:   0%|          | 0/315 [00:00<?, ?it/s]

Running Epoch 10 of 12:   0%|          | 0/315 [00:00<?, ?it/s]

Running Epoch 11 of 12:   0%|          | 0/315 [00:00<?, ?it/s]

(3780, 0.13608305006594507)

In [12]:
result, model_outputs, wrong_predictions = model.eval_model(validation_df)

  0%|          | 0/2234 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/280 [00:00<?, ?it/s]

In [13]:
predictions = model_outputs.argmax(axis=1)
actuals = validation_df['labels'].values

# Print classification report
print(metrics.classification_report(actuals, predictions, digits=3))

              precision    recall  f1-score   support

           0      0.766     0.764     0.765       577
           1      0.757     0.790     0.773       352
           2      0.920     0.910     0.915      1305

    accuracy                          0.854      2234
   macro avg      0.814     0.821     0.818      2234
weighted avg      0.855     0.854     0.854      2234

